In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import seaborn as sns
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from tensorflow import keras
import pymongo

In [2]:
def get_training():
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    db = client['plp']
    a= list(db["match"].find({}))
    a = pd.DataFrame(a)
    a = a.drop(columns = a.columns[0])
    return a

In [3]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")

def int_to_dummies(i):
    # Create a label for the number given
    #Input: int The value of the label
    result = [0]*3
    result[int(i)] = 1
    return np.array(result)

def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=[
                      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall'),
                      tf.keras.metrics.AUC(name='auc')])

    return model



def prediction_logits (y,batch_size,nb_label):
    # Normalize the batch of label to obtain a percentage
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    #Out : Numpy array [batch_size][10] the input normalized
    
    y_sum = np.sum(y, axis=1)
    for i in range (0,batch_size):
        for j in range (0,nb_label):
            if (y_sum[i] != 0):
                y[i][j] = y[i][j]/y_sum[i]
    
    return y


In [6]:
df = get_training()
df = df[df.ID_FTR != -1]
print(df)
target = df["ID_FTR"].apply(int_to_dummies)
features = df[df.columns[6:]]

     Div        Date               HomeTeam      AwayTeam FTR  ID_FTR  \
0     F1    20/09/14                 Nantes          Nice   H       2   
1     F1    21/09/14                  Lille   Montpellier   D       1   
2     F1    23/09/14                  Reims     Marseille   A       0   
3     F1    24/09/14                 Bastia        Nantes   D       1   
4     F1    24/09/14  Evian Thonon Gaillard          Lens   H       2   
...   ..         ...                    ...           ...  ..     ...   
4014  F2  10/01/2020                Le Mans       Auxerre   A       0   
4015  F2  10/01/2020                  Nancy  Valenciennes   H       2   
4016  F2  10/01/2020                  Niort      Le Havre   A       0   
4017  F2  10/01/2020                  Rodez   Chateauroux   A       0   
4018  F2  10/01/2020                Sochaux      Paris FC   D       1   

           AAG       AAS      AASO       AAF       AAC       AAY       AAR  \
0     0.262873  0.895837  0.375430  0.602974 

In [7]:
df

,Div,Date,HomeTeam,AwayTeam,FTR,ID_FTR,AAG,AAS,AASO,AAF,AAC,AAY,AAR,HAG,HAS,HASO,HAF,HAC,HAY,HAR
0,F1,20/09/14,Nantes,Nice,H,2,0.262873,0.895837,0.375430,0.602974,2.587834,-0.089255,-0.435110,-1.035393,0.953032,0.636636,0.907359,-0.103268,-0.417868,-0.429740
1,F1,21/09/14,Lille,Montpellier,D,1,-0.604166,0.692484,-0.006526,0.902012,0.493973,-0.089255,-0.435110,-0.172064,0.800350,0.636636,0.907359,1.388582,-0.417868,-0.429740
2,F1,23/09/14,Reims,Marseille,A,0,1.129912,1.709250,1.393978,1.250889,0.793096,-0.089255,-0.435110,-0.172064,0.138727,0.126082,0.608271,-0.700008,-0.755575,-0.429740
3,F1,24/09/14,Bastia,Nantes,D,1,-0.604166,1.200867,0.630067,1.201049,0.493973,-0.426711,-0.435110,0.259600,0.291409,0.253721,1.455686,-0.700008,1.270663,-0.429740
4,F1,24/09/14,Evian Thonon Gaillard,Lens,H,2,-0.170646,0.794161,0.630067,1.101370,1.092219,1.598025,-0.435110,-1.467057,0.240515,-0.512110,1.106751,0.493472,0.595250,-0.429740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,F2,10/01/2020,Le Mans,Auxerre,A,0,0.696392,0.692484,0.757385,1.250889,-0.702518,2.272936,1.584183,-0.172064,1.207502,1.530106,1.156598,1.090212,-0.080162,-0.429740
4015,F2,10/01/2020,Nancy,Valenciennes,H,2,1.129912,0.844999,0.630067,1.400408,1.092219,0.923113,-0.435110,-1.035393,0.342303,-0.129195,1.256294,0.493472,0.932957,-0.429740
4016,F2,10/01/2020,Niort,Le Havre,A,0,-0.604166,0.844999,0.502748,0.652813,-0.104273,0.585657,-0.435110,-0.172064,0.036939,0.253721,1.156598,-0.401638,1.608369,-0.429740
4017,F2,10/01/2020,Rodez,Chateauroux,A,0,-1.037685,0.336616,0.120793,1.250889,-0.702518,2.272936,1.584183,-0.603728,-0.013955,-0.001556,1.007055,0.195102,0.595250,1.626359


In [5]:
x_test = features.iloc[3550:].values
y_test = target.iloc[3550:].values

x_train = features.iloc[:3550].values
y_train = target.iloc[:3550].values

In [6]:
x_train = x_train.astype(float)

In [7]:
y_train = np.asarray(list(y_train))
y_test = np.asarray(list(y_test))

In [8]:
y_train.shape

(3550, 3)

In [9]:
model = create_model()

checkpoint_path = "models/model1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.fit(x_train,y_train, epochs=20,validation_data=(x_test,y_test),
          callbacks=[cp_callback])

Train on 3550 samples, validate on 427 samples
Epoch 1/20
3424/3550 [===========================>..] - ETA: 0s - loss: 1.0848 - accuracy: 0.6626 - precision: 0.4478 - recall: 0.0526 - auc: 0.5824
Epoch 00001: saving model to models/model1.ckpt
3550/3550 [==============================] - 5s 2ms/sample - loss: 1.0847 - accuracy: 0.6623 - precision: 0.4450 - recall: 0.0535 - auc: 0.5828 - val_loss: 1.1147 - val_accuracy: 0.6604 - val_precision: 0.3750 - val_recall: 0.0281 - val_auc: 0.5260
Epoch 2/20
3520/3550 [============================>.] - ETA: 0s - loss: 1.0702 - accuracy: 0.6650 - precision: 0.4828 - recall: 0.0716 - auc: 0.6036
Epoch 00002: saving model to models/model1.ckpt
3550/3550 [==============================] - 1s 164us/sample - loss: 1.0707 - accuracy: 0.6648 - precision: 0.4810 - recall: 0.0713 - auc: 0.6030 - val_loss: 1.1120 - val_accuracy: 0.6635 - val_precision: 0.4231 - val_recall: 0.0258 - val_auc: 0.5214
Epoch 3/20
3456/3550 [============================>.] - ETA

In [22]:
index = np.random.randint(0,107)
print(y_test[index])
print_logits(model.predict(x = x_test[index].reshape(1,14), batch_size = 1),1,3)

[0 0 1]
------------ 0 ------------
prédiction :  0 pourcentage de certitude : 32.3  %
prédiction :  1 pourcentage de certitude : 28.3  %
prédiction :  2 pourcentage de certitude : 39.5  %


In [24]:
y_pred = model.predict(x = x_train)
y_pred = np.argmax(y_pred, axis=1)
conf_mat = tf.math.confusion_matrix(y_train, y_pred)